In [1]:
import seaborn as sns
from queries import*
from utilities import*

## Importing Data

In [2]:
cell_data = pd.read_csv('data/cell_data.csv')

In [3]:
cell_data.head(3)

,sample_name,image_id,cell_x_position,cell_y_position,cell_id,tissue_category,phenotype,CD11C_mean,CD56_mean,CD3_mean,CD68_mean,SOX10_mean,CD20_mean,DAPI_mean,Autofluorescence_mean
0,DEEPMEL_1C1,14430_37727,14040.7,37383.4,1,stroma,DAPIp,0.107,0.060,0.068,0.973,0.049,0.073,11.844,3.879
1,DEEPMEL_1C1,14430_37727,14049.6,37381.9,2,stroma,MISSING,0.055,0.075,0.156,9.289,0.018,0.035,5.215,6.505
2,DEEPMEL_1C1,14430_37727,14056.5,37381.9,3,stroma,MISSING,0.018,0.019,0.020,0.708,0.008,0.041,9.428,4.085


In [5]:
cell_data['phenotype'] = cell_data['phenotype'].apply(lambda x: get_phenotype(x))

## EDA

In [5]:
cell_data.tissue_category.unique()

array(['stroma', 'tumor', 'missing'], dtype=object)

In [50]:
cell_data.phenotype.value_counts()

tumor          51559
stroma         11977
T                818
MISSING          769
macrophages      205
B                118
dendtritic        59
NK                29
Name: phenotype, dtype: int64

In [6]:
cell_data.tissue_category.value_counts()

tumor      58955
stroma      6559
missing       20
Name: tissue_category, dtype: int64

In [7]:
columns_to_keep = ['cell_id','cell_x_position','cell_y_position','tissue_category']
cell_data_restricted = cell_data[columns_to_keep]

## Computing pairwise distances

### Compute pairwise distances

In [8]:
UNIQUE_ID = cell_data_restricted.cell_id.unique()
N_CELL = len(cell_data_restricted.cell_id.unique())
N_REPARTITION = 30
repartition =[int(x) for x in np.linspace(0,N_CELL,N_REPARTITION)]

## All pairwise distances

In [9]:
def set_query(lower_bound_tab_1 =0, upper_bound_tab_1 = 1000, lower_bound_table_2 =0):
    
    query = '''SELECT
  distance,
  cell_id_1,
  cell_id_2
  tissue_category_1,
  tissue_category_2
FROM (
  SELECT
    table_1.cell_id_1,
    table_2.cell_id_2,
    table_1.x_1,
    table_1.y_1,
    table_2.x_2,
    table_2.y_2,
    tissue_category_1,
    tissue_category_2 ,

    SQRT(POW((x_1-x_2),2)+POWER((y_1-y_2),2)) AS distance
  FROM (
    SELECT
      cell_x_position AS x_1,
      cell_y_position AS y_1,
      cell_id AS cell_id_1,
      tissue_category as tissue_category_1
    FROM
      `advance-sonar-306410.deepmelo.cell_data`
      Where (cell_id>{lower1}) and  (cell_id<={upper1}) ) AS table_1
  CROSS JOIN (
    SELECT
      cell_x_position AS x_2,
      cell_y_position AS y_2,
      cell_id AS cell_id_2,
      tissue_category as tissue_category_2
    FROM
      `advance-sonar-306410.deepmelo.cell_data` 
      Where (cell_id>{lower2})) AS table_2 )'''.format(lower1=lower_bound_tab_1,upper1=upper_bound_tab_1,lower2=lower_bound_table_2)
    
    return query

## Bipartite graph query

In [5]:
def pairwise_distance_threshold(upper_thrd =100):
    
    query = '''SELECT
  distance,
  cell_id_1,
  cell_id_2,
  tissue_category_1,
  tissue_category_2
FROM (
  SELECT
    table_1.cell_id_1,
    table_2.cell_id_2,
    table_1.x_1,
    table_1.y_1,
    table_2.x_2,
    table_2.y_2,
    tissue_category_1,
    tissue_category_2 ,

    SQRT(POW((x_1-x_2),2)+POWER((y_1-y_2),2)) AS distance
  FROM (
    SELECT
      cell_x_position AS x_1,
      cell_y_position AS y_1,
      cell_id AS cell_id_1,
      tissue_category as tissue_category_1
    FROM
      `advance-sonar-306410.deepmelo.cell_data`
      Where (tissue_category = 'stroma' )) AS table_1
  CROSS JOIN (
    SELECT
      cell_x_position AS x_2,
      cell_y_position AS y_2,
      cell_id AS cell_id_2,
      tissue_category as tissue_category_2
    FROM
      `advance-sonar-306410.deepmelo.cell_data` 
      Where (tissue_category = 'tumor')) AS table_2 )

Where distance<{upper} '''.format(upper=upper_thrd)
    
    return query
    
    

## Extracting statistics from pairwise distances

### Defining a function running queries on Bigquery

In [6]:
def run_bq(query_name,return_df_csv = True):
    client = bigquery.Client() 
    
    query_job = client.query(queries[query_name])
    if return_df_csv:
        df = query_job.result().to_dataframe()
        df.to_csv('./data/output/'+query_name+ '.csv', index =False)
        print("Query results loaded to the table {} on VM".format(query_name))
        #return df
   

### Defining query names

In [7]:
### Queries names

BIPARTITE_QUERY = 'bipartite_tumor_stroma'
STATISTICS_STROMA = 'distance_statistics_to_stroma'
STATISTICS_TUMOR = 'distance_statistics_to_tumor'
STATISTICS_ALL_CELL = 'distance_statistics_all_cells'


### Running distance statistics queries

In [8]:
### statistics queries 
#statistics_all_cells = run_bq(STATISTICS_ALL_CELL)
#statistics_tumor = run_bq(STATISTICS_TUMOR)
#statistics_stroma = run_bq(STATISTICS_STROMA)

# Running bipartite Stroma vs Tumor

Fix a distance thereshold first that will determine the relational data base, we consider theresholds 20, 50, 80, 100, 200

### Getting pairwise distances by fixing cell_id lower and upper bounds

In [10]:
thrd_list = [150,200,250]

for thrd in thrd_list :
    table_name = 'pairwise_distances_thrd_{}'.format(thrd)
    client = bigquery.Client()
    query = pairwise_distance_threshold(thrd)
    df = client.query(query).to_dataframe()
    print('Dataframe with threshold {} extracted'.format(thrd))
    df.to_csv('./data/output/' + table_name + '.csv', index = False)

Dataframe with threshold 150 extracted
Dataframe with threshold 200 extracted
Dataframe with threshold 250 extracted


## Extracting edges with distance threshold 50

### Defining query

In [8]:
def query_distance_threshold(thrd=50):
    
    query = '''SELECT
    distance,
    cell_id_1,
  cell_id_2,
  tissue_category_1,
  tissue_category_2,
  phenotype_1,
  phenotype_2,
FROM (
  SELECT
    table_1.cell_id_1,
    table_2.cell_id_2,
    table_1.x_1,
    table_1.y_1,
    table_2.x_2,
    table_2.y_2,
    tissue_category_1,
    tissue_category_2 ,
    phenotype_1,
    phenotype_2,
    SQRT(POW((x_1-x_2),2)+POWER((y_1-y_2),2)) AS distance
    
  FROM (
    SELECT
      cell_x_position AS x_1,
      cell_y_position AS y_1,
      cell_id AS cell_id_1,
      tissue_category as tissue_category_1,
      phenotype as phenotype_1

    FROM
      `advance-sonar-306410.deepmelo.cell_data`
      Where (cell_id>0) ) AS table_1
  CROSS JOIN (
    SELECT
      cell_x_position AS x_2,
      cell_y_position AS y_2,
      cell_id AS cell_id_2,
      tissue_category as tissue_category_2,
      phenotype as phenotype_2,

    FROM
      `advance-sonar-306410.deepmelo.cell_data` 
      Where (cell_id>0)) AS table_2 )

 where distance < {thrd} and (cell_id_1 < cell_id_2)
      '''.format(thrd=thrd)
    return query

In [9]:
THRSHOLD = 100
table_name = 'edges_thrd_{}'.format(THRSHOLD)
client = bigquery.Client()
query = query_distance_threshold(THRSHOLD)
df = client.query(query).to_dataframe()
print('Dataframe with threshold {} extracted'.format(THRSHOLD))
df.to_csv('./data/output/' + table_name + '.csv', index = False)

Dataframe with threshold 100 extracted


In [10]:
len(df)

7887092

In [44]:
#!gsutil cp -r ./data/output/  gs://cell-deep-melo-lts4

# Extracting degrees

## Extracting degrees with threshold 50 and without border

In [6]:
total_edges = pd.read_csv('./data/output/edges_thrd_100.csv')

In [7]:
edges_data = graph_wrangling(total_edges, 50, False)
edges_data['phenotype_1'] = edges_data['phenotype_1'].apply(lambda x: get_phenotype(x))
edges_data['phenotype_2'] = edges_data['phenotype_2'].apply(lambda x: get_phenotype(x))

In [8]:
edges_data['is_border'] = edges_data.apply((lambda x : 1 if ( x.tissue_category_1 != x.tissue_category_2) else  0), axis = 1)

In [9]:
edges_data.head(3)

,distance,cell_id_1,cell_id_2,tissue_category_1,tissue_category_2,phenotype_1,phenotype_2,is_border
0,43.811528,6821,6894,stroma,stroma,T,stroma,0
8,27.397993,34746,34779,stroma,stroma,T,stroma,0
9,40.121441,35764,35871,stroma,tumor,T,tumor,1


## Extracing border cells

### Extracting the cells that are neighbours to the exact border

In [12]:
exact_border_df = edges_data[edges_data.is_border == 1].copy()
### Getting a list of cell_id  that are on the border
border_cells = get_cells_from_edges(exact_border_df)
### Fixing the radius of the neighberhood
border_radius = 30
### Getting all edges inside the area determined by the radius
border_neighbours = total_edges[total_edges.distance< border_radius].copy()
### keep only the edges having a border_cell ending point 
condition_1 = (border_neighbours['cell_id_1'].apply(lambda x: x in border_cells))|(border_neighbours['cell_id_2'].apply(lambda x: x in border_cells))
border_neighbours['is_border'] = condition_1



### Extracting a cell list with all border neighbors

In [13]:
border_neighbours_df = border_neighbours[border_neighbours['is_border']==True]
border_cells_neighbors = get_cells_from_edges(border_neighbours_df)

### The border column to the edges dataframe

In [14]:
print('The number of cells on the border', len(border_cells))
print('The number of cells in the ', border_radius ,' radius neighberhood',len(border_cells_neighbors))

The number of cells on the border 22123
The number of cells in the  30  neighberhood 35237


## Visualising border cells

In [16]:
cell_data_with_border = cell_data[['cell_id','cell_x_position','cell_y_position','tissue_category','phenotype']].copy()
cell_data_with_border['on_border'] = cell_data_with_border['cell_id'].apply(lambda cell: cell in border_cells_neighbors)

subject to tissue type

In [19]:
visualise_cells = cell_data_with_border[cell_data_with_border['on_border'] == True]
fig, ax = plt.subplots(figsize = (30,15))
fig = sns.scatterplot(data = visualise_cells , x='cell_x_position', y= 'cell_y_position', hue = 'tissue_category')
fig.savefig('./plots/scatterplot_cells_border_tissue.png')

subject to phenotype

In [18]:
fig, ax = plt.subplots(figsize = (30,15))
sns.scatterplot(data = visualise_cells , x='cell_x_position', y= 'cell_y_position', hue = 'phenotype')
fig.savefig('./plots/scatterplot_cells_border_phenotype.png')

Removing stroma cells 

In [61]:
#fig, ax = plt.subplots(figsize = (30,15))
#visualise_cells = visualise_cells[visualise_cells.phenotype != 'stroma']
#sns.scatterplot(data = visualise_cells , x='cell_x_position', y= 'cell_y_position', hue = 'phenotype')
#fig.savefig('./plots/scatterplot_cells_border_no_stroma.png')


### Adding neighbour cells to edge data

In [62]:
condition = (edges_data['cell_id_1'].apply(lambda x: x in border_cells_neighbors))|(edges_data['cell_id_2'].apply(lambda x: x in border_cells_neighbors))
edges_data['is_border'] = condition

## Saving nodes to csv

In [29]:
cell_data_with_border.to_csv('./data/nodes_with_border.csv', index = False)

## Saving edges to csv

In [63]:
edges_data.to_csv('./data/edges_border.csv', index = False)

## Neighbourhood on tissue level

In [132]:
#edges_data = remove_ordinary_stroma_cells(edges_data)

In [31]:
edges_data_grouped = edges_data.groupby(['cell_id_1','tissue_category_2']).agg('count')['cell_id_2'].reset_index()

In [32]:
edges_data_grouped.head(2)

,cell_id_1,tissue_category_2,cell_id_2
0,1,stroma,28
1,2,stroma,31


In [33]:
degree_table = pd.pivot_table(edges_data_grouped, values='cell_id_2', index=['cell_id_1'],
                    columns=['tissue_category_2'], aggfunc=np.sum)

degree_table.columns.name = None

In [34]:
degree_table.head(2)

,missing,stroma,tumor
cell_id_1,,,
1,NaN,28.0,NaN
2,NaN,31.0,NaN


In [35]:
degree_table = degree_table.reset_index().rename( columns = {'missing':'missing_degree','tumor':'tumor_degree','stroma':'stroma_degree'})

In [36]:
degree_table = degree_table.fillna(0)

In [37]:
degree_table.head(3)

,cell_id_1,missing_degree,stroma_degree,tumor_degree
0,1,0.0,28.0,0.0
1,2,0.0,31.0,0.0
2,3,0.0,32.0,0.0


In [38]:
degree_table['total_degree'] = degree_table.apply(lambda x :  + x.tumor_degree + x.stroma_degree , axis =1)

In [39]:
degree_table.head(3)

,cell_id_1,missing_degree,stroma_degree,tumor_degree,total_degree
0,1,0.0,28.0,0.0,28.0
1,2,0.0,31.0,0.0,31.0
2,3,0.0,32.0,0.0,32.0


## Neighbourhood on Phenotype level


In [40]:
phenotype_grouped = edges_data.groupby(['cell_id_1','phenotype_2']).agg('count')['cell_id_2'].reset_index()

In [41]:
phenotype_degree_table = pd.pivot_table(phenotype_grouped, values='cell_id_2', index=['cell_id_1'],
                    columns=['phenotype_2'], aggfunc=np.sum)

phenotype_degree_table.columns.name = None
phenotype_degree_table = phenotype_degree_table.fillna(0).reset_index()

In [42]:
phenotype_degree_table.head(3)

,cell_id_1,B,MISSING,NK,T,dendtritic,macrophages,stroma,tumor
0,1,0.0,3.0,0.0,0.0,0.0,0.0,25.0,0.0
1,2,0.0,2.0,0.0,0.0,0.0,0.0,29.0,0.0
2,3,0.0,1.0,0.0,0.0,0.0,0.0,31.0,0.0


## Appending all cell features together

In [43]:
cell_degree_grouped = degree_table.merge(phenotype_degree_table, on = 'cell_id_1', how = 'left')

In [44]:
cell_degree_grouped.head(3)

,cell_id_1,missing_degree,stroma_degree,tumor_degree,total_degree,B,MISSING,NK,T,dendtritic,macrophages,stroma,tumor
0,1,0.0,28.0,0.0,28.0,0.0,3.0,0.0,0.0,0.0,0.0,25.0,0.0
1,2,0.0,31.0,0.0,31.0,0.0,2.0,0.0,0.0,0.0,0.0,29.0,0.0
2,3,0.0,32.0,0.0,32.0,0.0,1.0,0.0,0.0,0.0,0.0,31.0,0.0


In [45]:
cell_degree_grouped.to_csv('./data/output/cell_degree.csv', index = False)

## Counting the occurence of edges

In [46]:
edge_phenotype_count = edges_data.groupby(['phenotype_1','phenotype_2']).agg('count')[['cell_id_1']]

In [47]:
edge_phenotype_count = edge_phenotype_count.reset_index()

In [48]:
edge_phenotype_count.head(3)

,phenotype_1,phenotype_2,cell_id_1
0,B,B,376
1,B,MISSING,15
2,B,NK,1


In [49]:
edge_phenotype_count_pivotted = pd.pivot_table(edge_phenotype_count, values='cell_id_1', index=['phenotype_1'],
                    columns=['phenotype_2'], aggfunc=np.sum)

edge_phenotype_count_pivotted.columns.name = None

edge_phenotype_count_pivotted = edge_phenotype_count_pivotted.fillna(0)

In [50]:
edge_phenotype_count_pivotted.head()

,B,MISSING,NK,T,dendtritic,macrophages,stroma,tumor
phenotype_1,,,,,,,,
B,376.0,15.0,1.0,1119.0,69.0,3.0,1354.0,771.0
MISSING,37.0,1240.0,5.0,119.0,0.0,38.0,3053.0,16332.0
NK,3.0,16.0,4.0,37.0,1.0,1.0,254.0,472.0
T,778.0,109.0,65.0,4429.0,322.0,78.0,8507.0,8490.0
dendtritic,37.0,6.0,9.0,342.0,98.0,5.0,708.0,346.0


In [51]:
count_matrix = edge_phenotype_count_pivotted.to_numpy()
diagonal_matrix = np.diag(np.diag(count_matrix))
transpose_matrix = count_matrix.transpose()
total = transpose_matrix + count_matrix - diagonal_matrix

In [52]:
columns = edge_phenotype_count_pivotted.columns
index = edge_phenotype_count_pivotted.index
edge_phenotype_count = pd.DataFrame(data = total, columns = columns, index = index)

In [53]:
edge_phenotype_count

,B,MISSING,NK,T,dendtritic,macrophages,stroma,tumor
phenotype_1,,,,,,,,
B,376.0,52.0,4.0,1897.0,106.0,16.0,2696.0,1517.0
MISSING,52.0,1240.0,21.0,228.0,6.0,63.0,5825.0,37273.0
NK,4.0,21.0,4.0,102.0,10.0,5.0,539.0,878.0
T,1897.0,228.0,102.0,4429.0,664.0,194.0,17189.0,16783.0
dendtritic,106.0,6.0,10.0,664.0,98.0,8.0,1452.0,528.0
macrophages,16.0,63.0,5.0,194.0,8.0,161.0,2739.0,4360.0
stroma,2696.0,5825.0,539.0,17189.0,1452.0,2739.0,94578.0,357793.0
tumor,1517.0,37273.0,878.0,16783.0,528.0,4360.0,357793.0,1477740.0


In [54]:
heatmap_df = edge_phenotype_count.drop(['MISSING'],axis=1)

In [57]:
#fig, ax = plt.subplots(figsize = (15,15))
#sns_fig = sns.heatmap(heatmap_df.iloc[:,:5], cmap=sns.color_palette("light:b", as_cmap=True)).get_figure()
#sns_fig.savefig('./plots/heatmap.png')